In [1]:
import sys

sys.path.append('../..')


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scienceplots
import torch
from tqdm import tqdm

from core.environment.env import Environment
from core.model.decision_network.network import DecisionNetwork


/Users/crinstaniev/opt/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
plt.style.use('science')

In [4]:
env = Environment()

In [5]:
model = DecisionNetwork(
    ts_dim=5,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
observation, info = env.reset()

In [7]:
observation.keys()

dict_keys(['image', 'description', 'ts_feature', 'ts', 'nft_wallet', 'usd_wallet'])

In [8]:
image_feature = observation['image']
text_feature = observation['description']
ts_feature = observation['ts_feature']
ts_data = observation['ts']
usd_wallet = observation['usd_wallet']
nft_wallet = observation['nft_wallet']


In [9]:
ts_data.shape

torch.Size([1, 16, 5])

In [10]:
action, percentage = model(
    ts_features=ts_feature,
    image_features=image_feature,
    text_features=text_feature,
    ts_data=ts_data,
    usd_wallet=usd_wallet,
    nft_wallet=nft_wallet,
)
action = torch.argmax(action, dim=1).item()
percentage = percentage.item()

In [11]:
percentage

1.0

In [12]:
action

0

In [21]:
device = torch.device("mps")

In [22]:
model = model.to(device)

In [26]:
observation, info = env.reset()
image_feature = observation['image']
text_feature = observation['description']
ts_feature = observation['ts_feature']
ts_data = observation['ts']
usd_wallet = observation['usd_wallet']
nft_wallet = observation['nft_wallet']

infos = []

for _ in tqdm(range(1000)):
    # agent policy that uses the observation and info
    with torch.no_grad():
        action, percentage = model(
            ts_features=ts_feature.to(device),
            image_features=image_feature.to(device),
            text_features=text_feature,
            ts_data=ts_data.to(device),
            usd_wallet=usd_wallet,
            nft_wallet=nft_wallet,
        )
    action = torch.argmax(action, dim=1).item()
    percentage = percentage.item()

    action = (action, percentage)

    # print('action:', action)

    observation, reward, terminated, truncated, info = env.step(action)

    image_feature = observation['image']
    text_feature = observation['description']
    ts_feature = observation['ts_feature']
    ts_data = observation['ts']
    usd_wallet = observation['usd_wallet']
    nft_wallet = observation['nft_wallet']

    infos.append(info)

    # print(info)

    if terminated:
        print(terminated)
        # break


  0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: Placeholder storage has not been allocated on MPS device!